In [ ]:
import requests, concurrent.futures, time
from threading import Lock
import pandas as pd
import numpy as nm



class Items:
  def __init__(self, itemIds, region = '10000002'):
    """
    Default Region: The Forge
    """
    print(f'-->Total items to Check: {len(itemIds)}')

    self.region = region
    self.itemIds = itemIds
    self.ItemInfoList = []

    self.dfItemInfo = pd.DataFrame()
    # self.dfItemInfoList = []

    self.HEADERS = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36',
                     'Accept-Language': 'en-US;q=0.9',
                     })


  # will run in Threads
  def ItemInfo(self, itemId):
    """
    itemId = List of max len 200
    """

    idS = str(itemId)[1:-1].replace(" ", "")

    # NO Region
    URL_NoRegion = f"https://api.evemarketer.com/ec/marketstat/json?typeid={idS}"

    # With Region
    URL_WithRegion = f"https://api.evemarketer.com/ec/marketstat/json?typeid={idS}&regionlimit={self.region}"


    res_nr = requests.get(URL_NoRegion, headers = self.HEADERS)
    res_r = requests.get(URL_WithRegion, headers = self.HEADERS)
    
    
    Allinfo = list(zip(res_nr.json(), res_r.json()))
    with lock:
      # print(res)
      # print(len(res.json()))
      # print("\n\n")
      self.ItemInfoList.extend(Allinfo)


  def chunks(self, lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


  def runThreading(self, MAX_THREADS):
    itemIdsChunks = list(self.chunks(self.itemIds, 199))

    threads = min(MAX_THREADS, len(itemIdsChunks))
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
      executor.map(self.ItemInfo, itemIdsChunks)


  def runThreadingForFinalList(self, threads):
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
      executor.map(self.makeItemInfoDF, self.ItemInfoList)


  # will run in Threads
  def makeItemInfoDF(self, itemInfoJSON):
    """send Item info JSON from JSON list"""
    a = itemInfoJSON
    
    # print(a[0], "\n", a[1], '\n\n\n')

    ItemID = a[1]['buy']['forQuery']['types'][0]
    # ItemName = dfTypeids.loc[dfTypeids['0'] == ItemID].values[0][1]

    sellPercentile = a[1]['sell']['fivePercent']
    sellMin = a[1]['sell']['min']
    sellAvg = a[1]['sell']['wavg']
    buyPercentile = a[1]['buy']['fivePercent']
    buyMax = a[1]['buy']['max']
    buyAvg = a[1]['buy']['wavg']

    sellPercentile_allRegion = a[0]['sell']['fivePercent']
    sellMin_allRegion = a[0]['sell']['min']
    

    if not sellAvg:
      return

    # Buy/Sell Avg Deff [wavg]
    df = pd.Series([buyAvg, buyPercentile]).pct_change() * 100
    # buyAvgDiff = format(df.values[1], ".3f")
    buyAvgDiff = df.values[1]

    df = pd.Series([sellAvg, sellPercentile]).pct_change() * 100
    # sellAvgDiff = format(df.values[1], ".3f")
    sellAvgDiff = df.values[1]
    
    # Percentile Diff
    df = pd.Series([buyPercentile, sellPercentile]).pct_change() * 100
    # percenDiff = format(df.values[1], ".3f")
    percenDiff = df.values[1]


    # Percentile Diff For [S2B] All to Current in %
    df = pd.Series([sellPercentile_allRegion, buyPercentile]).pct_change() * 100
    percenDiff_S2B = df.values[1]

    # Percentile Diff For [S2S] All to Current in %
    df = pd.Series([sellPercentile_allRegion, sellPercentile]).pct_change() * 100
    percenDiff_S2S = df.values[1]



    if nm.isfinite(percenDiff):

      URL = f"https://esi.evetech.net/latest/markets/{self.region}/history/?datasource=tranquility&type_id={ItemID}"


      res = requests.get(URL, headers = self.HEADERS)



      if res.ok and res.json():

        m13 = res.json()[-13:]

        # avgPrice13 = 0
        avgOrder13 = 0
        avgVolume13 = 0
        for m in m13:
          # avgPrice13 += m['average']
          avgOrder13 += m['order_count']
          avgVolume13 += m['volume']


        # avg order count in 13 days
        # avgOrder13 = format(avgOrder13/13, ".3f")
        avgOrder13 = avgOrder13/13

        # Stop if Order count is less then 300/day
        # if avgOrder13 < 299:
        #   return


        # sell price avg last 13 day
        # avgPrice13 = format(avgPrice13/13, ".3f")



        # inc/dec last 13 day
        # priceDiff13 = m13[-1]['average'] - m13[0]['average']
        # priceDiff13 = format(priceDiff13, ".3f")

        # inc/dec last 13 day in Percent
        df = pd.Series([m13[0]['average'], m13[-1]['average']]).pct_change() * 100
        priceDiffPercnt13 = format(df.values[1], ".3f")

        
        # avg sell volume
        # avgVolume13 = format(avgVolume13/13, ".3f")
        avgVolume13 = avgVolume13/13

        # Latest date for market history
        latestDate = m['date']
        

        # getting Name And Volume with ItemID
        URL = f"https://esi.evetech.net/latest/universe/types/{ItemID}/?datasource=tranquility&language=en"

        res = requests.get(URL, headers = self.HEADERS).json()
        
        ItemName = res['name']
        packaged_volume = res['packaged_volume']


        # with lock:
        #   print("Before Dict\n")
        #   print(type(percenDiff))
        #   print(percenDiff)
        #   print(sellAvgDiff)
        #   print(buyAvgDiff)
        #   print("\n\n\n")

        try:
          dict1 =  {
              'Item Name': ItemName,
              'Percentile Diff For All to Current (S2S)(%)' : round(percenDiff_S2S),
              'Percentile Diff For All to Current (S2B)(%)' : round(percenDiff_S2B),
              'Profit for All-R (S2S)(value)' : sellPercentile - sellPercentile_allRegion,
              'Profit for All-R (S2B)(value)' : buyPercentile - sellPercentile_allRegion,
              'Sell Percentile ALL Region' : sellPercentile_allRegion,
              'SellMin ALL Region' : sellMin_allRegion,
              'Sell Percentile': sellPercentile,
              'Buy Percentile': buyPercentile,

              'Order Count [13 days Avg]' : round(avgOrder13),
              'Items per Order [13 days Avg]' : round(float(avgVolume13)/float(avgOrder13)),
              'Total Sell Volume [13 days Avg]' : round(avgVolume13),  
              'Packaged Volume': packaged_volume,
              'Sell Min': sellMin,
              'Buy Max': buyMax,
              'Buy/Sell Diff (%)': round(percenDiff),
              'Sell Avg Diff (%)': round(sellAvgDiff),
              'Buy Avg Diff (%)': round(buyAvgDiff),
              'Profit Per Unit [same region]': sellPercentile - buyPercentile,
              # 'Avg Price [13 days]' : avgPrice13,
              'Price Diff (%) [from 13 day]' : priceDiffPercnt13,
              # 'Price Diff [13 day]' : priceDiff13,
              # 'ItemID': format(ItemID, ".0f"),
              'ItemID': ItemID,
              'latest Date [Market History]' : 'D: ' + str(latestDate),
          }
        except Exception as e:
          with lock:
            # print(type(percenDiff))
            print(percenDiff)
            print(avgOrder13)
            print(avgVolume13)
            print(sellAvgDiff)
            print(buyAvgDiff)
            print(e)
            print(ItemName, "\n\n")

            # print( e, "\nDict failed")


        # with lock:
        #   print("After Dict\n")
        with lock:
          # print("self.dfItemInfoList.append(dict)")
          # self.dfItemInfoList.append(dict1)
          self.dfItemInfo = self.dfItemInfo.append(dict1, ignore_index = True)
          





  def start(self, evemerketerThreads = 99, evetechThreads = 99):
    """Default Thread count 99 for All"""

    t0 = time.time()
    print("\nPlease wait a Moment...")
    self.runThreading(evemerketerThreads)
    print("-->Time Taken: ", (time.time() - t0)/60,'m' )


    t1 = time.time()
    print("\nMaking Final list with Threading...")
    self.runThreadingForFinalList(evetechThreads)

    print("-->Time Taken: ", (time.time() - t1)/60,'m', '\n\n')

    print("D O N E")
    print("-->Total Time Taken: ", (time.time() - t0)/60,'m', '\n\n')

    # return self.ItemInfoList
    # return self.ItemInfoList
    print("Total items after web Request: ", len(self.ItemInfoList))
    print("Final list: ", len(self.dfItemInfo), "\n\n")
    # print("dfItemInfoList: ", len(self.dfItemInfoList), '\n\n\n')         



# if __name__ == "__main__":
print("Starting...")

print("Collecting All ItemsID and Names...\n\n")

dfTypeids = pd.read_csv('http://www.fuzzwork.co.uk/resources/typeids.csv')
itemIds = list(dfTypeids['0'].values)


lock = Lock()

# itemsInfo = Items(itemIds=itemIds[4450:4500])
itemsInfo = Items(itemIds=itemIds)

itemsInfo.start(199, 37)

Starting...


-->Total items to Check: 43049

Please wait a Moment...
-->Time Taken:  1.2633331457773844 m

Making Final list with Threading...
-->Time Taken:  13.110769712924958 m 


D O N E
-->Total Time Taken:  14.374114326635997 m 


Total items after web Request:  43049
Final list:  12779 




In [ ]:
# No Region
itemsInfo.dfItemInfo.to_csv("/content/drive/MyDrive/00_Sheets/EVE_AllToJita.csv", encoding = "utf-8",  index=False)

In [ ]:
itemsInfo.dfItemInfo.tail()

In [ ]:
itemsInfo.dfItemInfo

Item Name  Percentile ALL Region  SellMin ALL Region  \
0   Particle Accelerator Unit               16425.80             16410.0   
1     Laser Focusing Crystals               22302.32             15000.0   
2        Superconductor Rails               17988.34             15000.0   
3  Thermonuclear Trigger Unit               18740.00             18740.0   
4      Plasma Pulse Generator               19025.40              5000.0   
5    Graviton Pulse Generator               18960.04               214.0   
6     Nuclear Pulse Generator               17129.77              5000.0   
7          EM Pulse Generator               18118.14             15000.0   

   Percentile Diff For All to Current (S2S)(%)  \
0                                          0.0   
1                                          0.0   
2                                         17.0   
3                                          0.0   
4                                          0.0   
5                                          0.0   
6                                          0.0   
7                                          1.0   

   Percentile Diff For All to Current (S2B)(%)  \
0                                         -3.0   
1                                         -1.0   
2                                          4.0   
3                                         -9.0   
4                                        -22.0   
5                                        -15.0   
6                                         -9.0   
7                                        -12.0   

   Percentile Diff For All (S2S)(value)  Percentile Diff For All (S2B)(value)  \
0                                   4.0                                -416.0   
1                                  38.0                                -292.0   
2                                3002.0                                 632.0   
3                                   0.0                               -1640.0   
4                                 -25.0                               -4134.0   
5                                  30.0                               -2750.0   
6                                   1.0                               -1500.0   
7                                 135.0                               -2118.0   

   Order Count [13 days Avg]  Items per Order [13 days Avg]  \
0                       77.0                          186.0   
1                       48.0                          760.0   
2                       24.0                          594.0   
3                       38.0                          537.0   
4                       11.0                          574.0   
5                       19.0                         1280.0   
6                        7.0                          962.0   
7                       14.0                         1013.0   

   Total Sell Volume [13 days Avg]  ...  Buy Percentile  Sell Min  Buy Max  \
0                          14300.0  ...         16010.0   16430.0  16010.0   
1                          36210.0  ...         22010.0   22340.0  22010.0   
2                          14083.0  ...         18620.0   20990.0  18620.0   
3                          20521.0  ...         17100.0   18740.0  17100.0   
4                           6493.0  ...         14891.7   19000.0  15000.0   
5                          24718.0  ...         16210.0   18990.0  16210.0   
6                           6884.0  ...         15630.0   17000.0  15630.0   
7                          13793.0  ...         16000.0   17500.0  16000.0   

   Buy/Sell Diff (%)  Sell Avg Diff (%)  Buy Avg Diff (%)  Profit Per Unit  \
0                3.0               -9.0              11.0           420.00   
1                1.0              -11.0              17.0           330.00   
2               13.0              -16.0              32.0          2370.00   
3               10.0               -6.0              20.0          1640.00   
4               28.0              -11.0   

In [ ]:
df = pd.Series([10, 0]).pct_change() * 100

# df.values[1]
df

0      NaN
1   -100.0
dtype: float64

In [ ]:
a = [1,2,3]
b = [4,5,6]

In [ ]:
z = zip(a,b)

In [ ]:
list(z)

[(1, 4), (2, 5), (3, 6)]